# TP 3 - Opérations locales

Dans ce TP, nous allons traiter les images à une échelle locale. Il s'agit de considérer l'image par "régions".

Ce TP est **noté**. Vous pouvez rendre votre code en m'envoyant le fichier Jupyter (format `ipynb`) par mail à `florent.grelard [at] u-bordeaux.fr`.

Vous répondrez aux questions "ouvertes" ("Que constatez-vous...?", "Pourquoi...?", etc.) dans le TP par des commentaires dans votre code Python. Toutes tentatives et justifications pertinentes seront valorisées.

Vous pouvez vous aider de toutes les fonctions numpy/skimage existantes. Si vous êtes bloqués à une question, vous pouvez utiliser les fonctions de ces bibliothèques pour passer aux questions suivantes.

## 1. Opérateurs de morphologie mathématique

Dans ce premier exercice, l'objectif est d'extraire une image de contours en utilisant les opérateurs de morphologie mathématique. Dans un premier temps, nous allons implémenter les opérateurs d'**érosion** et de **dilatation**. L'image de contours sera donnée par le **gradient morphologique**, défini par $I_{\text{dilatée}} - I_{\text{érodée}}$.

<table>
    <tr>
    <td>
        <img src="data/obj_morphomaths.png" width="700" /> 
        <figcaption> De gauche à droite: (a) Image originale, (b) Image érodée avec une taille de 10, (c) Image dilatée avec une taille de 10, (d) Image de gradient avec une taille de 1 </figcaption>
    </td>
    </tr>
</table>

Les opérateurs de morphologie mathématique classiques fonctionnent sur une image binaire. Une région, appelée *élément structurant*, est positionnée en chaque pixel et détermine la nouvelle appartenance à l'objet ou à l'arrière-plan en fonction de l'opérateur:
* pour l'**érosion**: pour qu'un pixel soit objet, tous les pixels dans la région doivent être objet.
* pour la **dilatation**: pour qu'un pixel soit objet, au moins un pixel dans la région doit être objet.

**Exercice**: Compléter le code suivant:
1. Compléter la fonction `extract_region` qui extrait la sous-image à la position (x,y) pour une région de rayon `size`.
2. Compléter les fonctions `erosion` et `dilatation`. Afficher les images obtenues. 
3. Utiliser une taille d'élement structurant égale à 10. Que constatez-vous? 
4. Résoudre le problème précédent en employant la méthode de votre choix. Vous pouvez vous aider des supports de cours ainsi que de la documentation numpy. Préciser la méthode employée en commentaire.
5. Calculer le gradient morphologique (fonction `morphological_gradient`) et afficher le résultat.
6. Comparer vos résultats avec ceux obtenus par les fonctions de la bibliothèque `skimage.morphology`.

In [9]:
%matplotlib qt5
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage import color

def extract_region(image, x, y, size):
    """
    Extrait la sous-matrice à la position (x,y)
    pour une région de taille `size`
    """
    return image[x-size: x+size+1, y-size: y+size+1]

def erosion(image, size):
    image_copy = np.pad(image, size)
    image_out = image_copy.copy()
    for x,y in np.ndindex(image.shape):
        x_n = x+size
        y_n = y+size
        region = extract_region(image_copy, x_n, y_n, size)
        if region.all():
            image_out[x_n, y_n] = 255
        else:
            image_out[x_n, y_n] = 0
    image_out = image_out[size:-size, size:-size]
    return image_out

def dilatation(image, size):
    image_copy = np.pad(image, size)
    image_out = image_copy.copy()
    for x,y in np.ndindex(image.shape):
        x_n = x+size
        y_n = y+size
        region = extract_region(image_copy, x_n, y_n, size)
        if region.any():
            image_out[x_n, y_n] = 255
        else:
            image_out[x_n, y_n] = 0
    image_out = image_out[size:-size, size:-size]
    return image_out

def morphological_gradient(image, size):
    eroded = erosion(image, size)
    dilated = dilatation(image, size)
    return dilated - eroded

img_disk = io.imread("data/disquette.jpg")

# On seuille pour obtenir une image binaire
img_disk = np.where(img_disk > 127, 0, 255).astype(np.uint8)

size = 1

img_eroded = erosion(img_disk, 10)
img_dilation = dilatation(img_disk, 10)
img_gradient = morphological_gradient(img_disk, size)
img_diff = img_disk - img_eroded
fig, ax = plt.subplots(1, 4)

ax[0].imshow(img_disk, cmap="gray")
ax[1].imshow(img_eroded, cmap="gray")
ax[2].imshow(img_dilation, cmap="gray")
ax[3].imshow(img_gradient, cmap="gray")
[axi.set_axis_off() for axi in ax]

[None, None, None, None]

[None, None, None, None]

## 2. Filtres et convolution

Les filtres sont des régions carrées centrées en un pixel avec des coefficients variables afin, par exemple, de flouter l'image, de réduire le bruit, ou encore d'extraire le contour des objets. 

La convolution consiste en la somme des produits des coefficients du filtre $F$ avec les intensités originales de l'image $I$ dans un voisinage $V$ : 

$
    I'(x, y) = \sum_{(i, j) \in V(x, y)} I(i, j) * F(i, j)
$

Nous cherchons à débruiter l'image afin d'obtenir le résultat ci-dessous:

<table>
<tr>
<td>
<img src="data/obj_bruit.png" width="500"> 
<figcaption style="text-align:center;"> De gauche à droite: (a) Image originale, (b) Image débruitée </figcaption>
</td>
</tr>
</table>

Nous allons implémenter le filtre moyen, ainsi que le filtre médian.

**Exercice**: Compléter le code suivant:
1. Compléter la fonction `mean_filter_coefficients` pour générer une région de taille $n \times n$ avec les coefficients du filtre moyen. On rappelle que les coefficients du filtre moyen de taille $n \times n$ sont tous égaux à $\frac{1}{n^2}$.
2. Compléter la fonction `convolution` qui permet de calculer une nouvelle intensité par convolution d'une région de l'image et d'un filtre de taille $n \times n$.
3. Compléter la fonction `generic_filtering` qui permet de calculer une image filtrée par convolution avec un filtre quelconque. Cette fonction appelle la fonction `convolution` sur l'ensemble de l'image.
4. Utiliser la fonction `generic_filtering` pour appliquer un filtrage moyen.
5. Compléter la fonction `median_filtering` pour calculer le filtrage médian. Cette fonction n'utilise pas la fonction `convolution` puisqu'elle n'utilise pas de filtre avec des coefficients.
6. Comparer les résultats obtenus avec les filtres moyen et médian. Quel filtre semble plus adapté à réduire le bruit? Pourquoi?
7. Comparer vos résultats avec ceux obtenus par les fonctions du module `skimage.filters.rank`.

In [102]:
def mean_filter_coefficients(size):
    """
    Filtre de taille size x size
    """
    return np.ones((size, size))/size**2

def convolution(region, filtre):
    """
    Convolution d'une région de l'image
    avec un filtre

    Prérequis: `region` et `filtre` doivent être de la même taille


    Parameters
    ----------
    region: np.ndarray
        nxn region
    filtre: np.ndarray
        nxn filter with coefficients

    Returns
    -------
    new_intensity: float
        la nouvelle intensité obtenue par convolution
    """
    new_intensity = 0
    for (x, y) in np.ndindex(region.shape):
        new_intensity += region[x,y] * filtre[x, y]
    return new_intensity

def generic_filtering(image, filtre):
    """
    Convolution avec filtre moyen

    Parameters
    ----------
    image: np.ndarray
        image
    filtre: np.ndarray
        filtre de taille nxn

    Returns
    -------
    image: np.ndarray
        image filtrée
    """
    size = filtre.shape[0]
    radius = size//2
    image_copy = np.pad(image, radius)
    image_out = image_copy.copy()
    for (x, y) in np.ndindex(image.shape):
        x += radius
        y += radius
        region = extract_region(image_copy, x, y, radius)
        new_intensity = convolution(region, filtre)
        image_out[x, y] = new_intensity
    return image_out[radius:-radius, radius:-radius]

def median_filtering(image, size):
    """
    Application du filtrage médian

    Parameters
    ----------
    image: np.ndarray
        image
    size: int
        taille de la région (size * size)

    Returns
    -------
    image: np.ndarray
        image filtrée
    """
    radius = size//2
    image_copy = np.pad(image, radius)
    image_out = image_copy.copy()
    for (x, y) in np.ndindex(image.shape):
        x += radius
        y += radius
        region = extract_region(image, x, y, radius)
        image_out[x, y] = np.median(region)
    return image_out[radius:-radius, radius:-radius]

img_noisy = io.imread("data/bruit.jpg")
mean_coeffs = mean_filter_coefficients(5)
img_mean = generic_filtering(img_noisy, mean_coeffs)
img_median = median_filtering(img_noisy, 5)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(img_noisy, cmap="gray")
ax[1].imshow(img_median, cmap="gray")
[axi.set_axis_off() for axi in ax]

[None, None]

## 3. Algorithme de Canny

Le but de cet exercice et d'implémenter l'algorithme de Canny. L'algorithme de Canny permet d'extraire des contours sur des images en niveaux de gris.

<table>
<tr>
<td>
<img src="data/obj_canny.png" width="600"> 
<figcaption style="text-align:center;"> De gauche à droite: (a) Image originale, (b) Image de contours </figcaption>
</td>
</tr>
</table>

Voici ses étapes:
1. Lissage de l'image par un filtre gaussien (ou moyen).
2. Extraction du gradient par les **filtres de Sobel**.
3. Extraction des **maxima locaux**: on garde les pixels dans l'image uniquement si leur valeur est supérieure à celle de leurs voisins, sinon on les met à 0.
4. On seuille l'image de contours pour produire une image binaire.

**Exercice**: Compléter le code suivant
1. Lisser l'image. Il est fortement recommandé d'utiliser un **filtre gaussien**. Les coefficients du filtre gaussien sont obtenus par la formule suivante:

\begin{equation}
    \dfrac{1}{2 \pi \sigma^2} . e^{-\dfrac{x^2 + y^2}{2 \sigma^2}}
\end{equation}

où $\sigma$ est l'écart-type de la gaussienne, et $x$ et $y$ les coordonnées dans la région de taille $n \times n$.

Compléter et appliquer la fonction `gaussian_coefficients`. On pourra également utiliser le filtre moyen précédemment implémenté.

2. Compléter la fonction `sobel_coefficients`. On rappelle que les filtres sont définis par : 

\begin{equation}
S_h =
\begin{bmatrix}
    - 1 & -2 & -1\\
    0 & 0 & 0 \\
    1 & 2 & 1
\end{bmatrix}\quad \text{et} \quad
S_v = 
\begin{bmatrix}
    - 1 & 0 & 1\\
    -2 & 0 & 2 \\
    -1 & 0 & 1
\end{bmatrix}
\end{equation}

3. Effectuer la convolution de l'image avec les filtres de Sobel horizontaux et verticaux. 

4. Calculer l'image de magnitude des gradients dans la fonction `magnitude`.
5. Extraire les maxima locaux (fonction `extract_local_maxima`) en comparant l'intensité de chaque pixel par rapport à ses voisins horizontaux et verticaux, **séparément**. Si la valeur du pixel est supérieure à celle de ses voisins, on conserve l'intensité originale, sinon on la remplace par zéro.
6. Seuiller l'image des maxima locaux afin d'obtenir une image de contours binaire.
7. Commentez le résultat obtenu: quel seuil choisir?
8. Comparez vos résultats avec les fonctions des modules `skimage.filters` et `skimage.features.canny`.

In [ ]:
def gaussian_coefficients(size, sigma):
    """
    Coefficients de la gaussienne 2D

    Parameters
    ----------
    size: int
        taille n x n du filtre
    sigma: int
        écart type de la gaussienne

    Returns
    -------
    filter: np.ndarray
        filtre n x n avec les coefficients
    """
    coefficients = np.zeros((size, size))
    for x, y in np.ndindex(coefficients.shape):
        coefficients[x, y] = 1 / (2 * np.pi * sigma ** 2) * np.exp(-(x**2+y**2)/(2*sigma**2))
    return coefficients

def sobel_coefficients():
    """
    Retourne un matrice 3x3 avec les 
    coefficients de Sobel:
    [-1 -2 -1
      0  0  0
      1  2  1]
    """
    return np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

def magnitude(image_1, image_2):
    """
    Magnitude de deux images
    definie comme sqrt(image_1^2 + image_2^2)
    """
    return np.sqrt(image_1**2 + image_2**2)

def extract_local_maxima(image, size):
    """
    Extrait les maxima locaux d'une image
    cad les pixels qui ont l'intensité la plus élevée
    dans un voisinage. Si l'intensité n'est pas maximale par
    rapport aux voisins, l'intensité est mise à zéro.

    Parameters
    ----------
    image: np.ndarray
        image
    size: int
       taille du voisinage (n x n)

    Returns
    -------
    image: np.ndarray
        image des maxima locaux
    """
    radius = size//2
    image_copy = np.pad(image, radius)
    image_out = image_copy.copy()
    for (x, y) in np.ndindex(image.shape):
        x += radius
        y += radius
        region = image_copy[x-radius:x+radius+1, y]
        region2 = image_copy[x, y-radius:y+radius+1]
        intensity = image_copy[x, y]
        if region.max() != intensity and region2.max() != intensity:
            image_out[x, y] = 0
    return image_out[radius:-radius, radius:-radius]

img_leaves = io.imread("data/leaves.jpg")
img_leaves = color.rgb2gray(img_leaves)*255

size = 3
sigma = 10
coeffs = gaussian_coefficients(size, sigma)
sobel_coeffs = sobel_coefficients()

img_smooth = generic_filtering(img_leaves, coeffs)
img_h = generic_filtering(img_smooth, sobel_coeffs)
img_v = generic_filtering(img_smooth, sobel_coeffs.T)
img_gradient = magnitude(img_h, img_v)
img_local = extract_local_maxima(img_gradient, 3)
img_threshold = np.where((img_local > 1.5), 255, 0)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(img_smooth, cmap="gray")
ax[1].imshow(img_threshold, cmap="gray")
[axi.set_axis_off() for axi in ax]

[None, None]

## 4. Effet pointilliste: images RGB

On cherche ici à implémenter l'effet pointilliste tel que montré sur la figure ci-dessous:

<table>
<tr>
<td>
<img src="data/pointillism_1.png" width="400"> 
<figcaption style="text-align:center;">Image originale </figcaption>
</td>
<td>
<img src="data/pointillism_3.png" width="400"> 
<figcaption style="text-align:center;"> Image avec effet pointilliste </figcaption>
</td>
</tr>
</table>

Le principe est de dessiner, en chaque pixel, un cercle dont la couleur correspond à la couleur moyenne du pixel prise dans un voisinage.

Il s'agit donc d'adapter le code précédent aux images couleur.

**Exercice**: Compléter le code suivant:
1. Compléter la fonction `extract_region_rgb` pour extraire la région de taille `size` dans une image RGB. Elle doit retourner une sous-région avec 3 dimensions (hauteur, largeur, canaux).
2. Compléter la fonction `mean_color`: pour un pixel, extraire la couleur moyenne dans une région de taille `size`. On pourra s'aider des fonctions définies à l'exercice précédent.
3. Dessiner 100.000 disques à l'aide de la fonction `draw.disk` de `skimage`. Les disques auront une taille et une position variables au sein de l'image (s'aider de la fonction `numpy.random.randint`). Leur couleur sera déterminée à l'aide de la fonction `mean_color`.
On conseille d'alterner le rayon du disque entre 3, 5 et 7.

In [206]:
from skimage import draw

def extract_region_rgb(image_rgb, x, y, size):
    """
    Extrait la sous-matrice à la position (x,y)
    pour une région de taille `size` pour une image
    RGB

    Returns
    -------
    region: np.ndarray
        region avec 3 dimensions (hauteur, largeur, nombre canaux)
    """
    return image_rgb[x-size: x+size+1, y-size:y+size+1, :]

def mean_color(image, x, y, size):
    """
    Extrait la couleur moyenne en (x,y) pour
    une image RGB

    Returns
    -------
    rgb: list
        liste avec 3 éléments: moyennes pour chaque canal
    """
    if size % 2 == 0:
        size += 1
    region = extract_region_rgb(image, x, y, size//2)
    filtre = mean_filter_coefficients(size)
    r = convolution(region[..., 0], filtre)
    g = convolution(region[..., 1], filtre)
    b = convolution(region[..., 2], filtre)
    return (r, g, b)

def pointillism(image, size):
    """
    Création d'un effet pointilliste

    Génération de plusieurs disques dont la couleur est la
    couleur moyenne prise dans un voisinage de taille
    et de positions variables
    """
    radius = size//2
    image_copy = np.pad(image, [(radius, radius), (radius, radius), (0, 0)])
    image_out = image_copy.copy()
    for i in range(int(1e5)):
        rad = np.random.randint(3, 7)
        x = np.random.randint(0, image.shape[0])
        y = np.random.randint(0, image.shape[1])
        rr, cc = draw.disk((rad, rad), rad)
        rd = rr + x
        cd = cc + y
        x += radius
        y += radius
        image_out[rd, cd, :] = mean_color(image_copy, x, y, rad)
    return image_out[radius:-radius, radius:-radius]

img_corals = io.imread("data/pointillism_1.png")

size = 14

img_out = pointillism(img_corals, size)
plt.imshow(img_out)
plt.axis("off")

(-0.5, 641.5, 357.5, -0.5)

## 5. Bonus: moyennes non-locales

La convolution classique permet de traiter le bruit de type "poivre et sel". En revanche, elle ne permet pas de corriger les images entachées d'autres types de bruit. C'est notamment le cas du bruit gaussien:

<table>
<tr>
<td>
<img src="data/tabu_2_noisy.png" width="400"> 
<figcaption style="text-align:center;"> Image avec bruit gaussien </figcaption>
</td>
<td>
<img src="data/tabu_2_noisy_median.png" width="400"> 
<figcaption style="text-align:center;"> Image débruitée avec filtre médian </figcaption>
</td>
<td>
<img src="data/obj_nlmeans.png" width="400"> 
<figcaption style="text-align:center;">Image débruitée avec moyennes non-locales </figcaption>
</td>
</tr>
</table>

On remarque que le **filtre médian gomme des détails importants** de l'image.

Nous allons choisir les **méthodes non-locales** qui permettent de parvenir à un résultat intéressant.
Les méthodes non-locales (non-local means) utilisent la redondance de l'information dans
l'image afin, par exemple, de réduire le bruit ([Wikipédia](https://en.wikipedia.org/wiki/Non-local_means), [Article original](http://www.ipol.im/pub/art/2011/bcm_nlm/)).

Elles calculent la similarité entre deux points dans l'image en considérant
des voisinages, appelés *patchs*, autour de ces points. Les
patchs correspondent à des voisinages carrés centrés en un point, de
taille $n \times n$.

Soient $I_{N(p)}(i)$ l'intensité du point $i$ dans le patch
$N$ centré en $q$, $I_{N(q)}(i)$ l'intensité du point $i$ dans le
patch $N$ centré en $q$, et $\Delta(N_p, N_q)_i =  I_{N(p)}(i) - I_{N(q)}(i)$
la différence d'intensité entre $N_p(i)$ et $N_q(i)$. La similarité
$w(p, q)$ entre $p$ et $q$ est donnée par :

\begin{equation}
  w(p, q) = e^{-\dfrac{\sqrt{\frac{1}{n^2} \sum_{i \in N} \Delta(N_p,
        N_q)_i^2}}{h}},
\end{equation}
où $h$ est un facteur de régularisation.

Pour le débruitage d'un point $p$ dans l'image, les valeurs de
similarité sont calculées dans un voisinage carré $V$ de $p$. La nouvelle intensité de $p$ est alors
donnée par la moyenne des intensités pondérée par les valeurs de similarité :
\begin{equation}
  I'(p) = \dfrac{1}{\sum{v in \V} w(p, v)} \sum_{v \in V} w(p, v) I(v)
\end{equation}

<table>
<tr>
<td>
<img src="data/nlmeans.png" width="400"> 
<figcaption style="text-align:center;"> Principe du débruitage par patchs: la nouvelle intensité au centre du patch $P$ est donnée comme une somme des intensités prises dans un voisinage $V$ pondérées par leur similarité par rapport à $P$. Ici, le patch $N_2$ a une plus forte similarité que $N_1$ vis-à-vis de $P$ (c'est-à-dire que ses intensités sont plus proches de celles de $P$).</figcaption>
</td>
</tr>
</table>

**Exercice**:
1. Compléter la fonction `similarity` qui calcule la similarité entre deux patchs `region1` et `region2`. Cette fonction correspond à la formule $w(p, q)$ ci-dessus.
2. Compléter la fonction `nlmeans` qui effectue le débruitage par moyennes non-locales. Pour chaque pixel: on extrait la région $V$ (grand voisinage), puis, dans ce voisinage, on extrait les régions $R$ (patchs) pour calculer la similarité avec le patch courant.
3. La complexité de ce calcul est élevée. Afin de réduire le temps de calcul, utilisez la bibliothèque `numba` (décommentez la première ligne du bloc de code), et décorez vos fonctions en ajoutant `@jit` au-dessous de leur signature (au-dessus de `def`). Cette instruction permet de compiler ces fonctions, et de réduire drastiquement leur temps d'exécution.
La plupart des fonctions de `numpy` sont reconnues par `numba`. **Attention** toutefois, **ce n'est pas le cas pour toutes** (par exemple `numpy.pad`).
Redéfinissez les fonctions déjà existantes que vous utilisez, telles que `extract_region`, en les décorant par `@jit`.
4. Tester votre code optimisé sur une image plus grosse, telle que `data/tabu_2_noisy.png`.
5. Comparer les résultats obtenus avec la fonction `skimage.restoration.denoise_nl_means`.

In [6]:
from numba import njit, jit

# @jit
def zero_pad(image, padding):
    """
    Zero padding pour numba
    """
    padded = np.zeros((image.shape[0] + 2*padding, image.shape[1] + 2*padding))
    padded[padding:-padding, padding:-padding] = image
    return padded

# @jit
def extract_region_clip(image, x, y, size):
    """
    Extraction de la région pour numba
    """
    r = size//2
    xmin = x-r
    xmax = x+r+1
    ymin = y-r
    ymax = y+r+1
    return image[xmin:xmax, ymin:ymax]

# @jit
def similarity(region1, region2, h):
    """
    Calcul la similarité entre deux 
    patchs region1 and region2

    Parameters
    ----------
    region1: np.ndarray
        r x r patch
    region2: np.ndarray
        r x r patch
    h: int
        facteur lié à l'écart-type du bruit dans l'image

    Returns
    -------
    similarity: float
        la similarité entre region1 et region2
    """
    sum_diff = 0
    for x in range(region1.shape[0]):
        for y in range(region1.shape[1]):
            sum_diff += np.square(region1[x, y] - region2[x,y])
    exp_factor = np.sqrt(sum_diff / region1.size**2) / h
    return np.exp(-exp_factor)

# @jit
def nlmeans(image, v_size, r_size, h):
    """
    Calcul des moyennes non locales
    pour débruiter l'`image`

    Parameters
    ----------
    image: np.ndarray
        l'image à débruiter
    v_size: int
        taille du grand voisinage V
    r_size: int
        taille du petit voisinage r
        (= taille du patch)
    h: int
        facteur lié à l'écart-type du bruit

    Returns
    -------
    image: np.ndarray
        image débruitée
    """
    v_r = v_size//2
    r_r = r_size//2
    padding = v_r+r_r
    image_copy = zero_pad(image, padding)
    image_out = image_copy.copy()
    for x in range(padding, image_copy.shape[0]-padding):
        for y in range(padding, image_copy.shape[1]-padding):
            current_patch = extract_region_clip(image_copy, x, y, r_size)
            new_intensity = 0
            s_sum = 0
            for i in range(v_size):
                for j in range(v_size):
                    other_x, other_y = x+i-v_r, y+j-v_r
                    other_patch = extract_region_clip(image_copy, other_x, other_y, r_size)
                    s = similarity(current_patch, other_patch, h)
                    s_sum += s
                    new_intensity += s * image_copy[other_x, other_y]
            image_out[x, y] = new_intensity / s_sum
    return image_out[padding:-padding, padding:-padding]

img_nlmeans = io.imread("data/tabu_2_noisy.png")
img_nlmeans = io.imread("data/tabu_2_closeup.png")
print(img_nlmeans.shape)
#Taille du grand voisinage : v_size x v_size
v_size = 20

#Taille du patch : r_size x r_size
r_size = 7

#Facteur h
h = 1.25

img_denoised = nlmeans(img_nlmeans, v_size, r_size, h)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(img_nlmeans, cmap="gray")
ax[1].imshow(img_denoised, cmap="gray")
[axi.set_axis_off() for axi in ax]

(102, 120)


[None, None]